In [17]:
import sys
import warnings
import time

from tqdm import tqdm
from bs4 import BeautifulSoup

import pyderman

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

from pprint import pprint

import json

In [2]:
def open_chrome(headless=True):
    path = pyderman.install(browser=pyderman.chrome)

    options = Options()
    options.add_argument("--enable-javascript")
    options.headless = headless

    driver = webdriver.Chrome(path, options=options)

    # Sanity checks.
    driver.get("http://www.python.org")
    assert "Python" in driver.title
    return driver



In [3]:
driver = open_chrome(False)

chromedriver is already installed.


In [4]:
gscholar_url = "https://scholar.google.com/citations?user=qre6vl0AAAAJ&hl=en"
driver.get(gscholar_url)

In [5]:
while True:
    e = driver.find_element_by_xpath('//button[@id="gsc_bpf_more"]')
    e.click()
    if e.get_attribute('disabled') == 'true':
        break

In [18]:
publications = []

for e in driver.find_elements_by_xpath('//tr[@class="gsc_a_tr"]'):
    e.click()
    _json = {}
    time.sleep(0.7)
    try:
        bsoup = BeautifulSoup(driver.page_source.encode('utf-8'))
        cite = bsoup.find('div', attrs={'id': 'gs_md_cita-l'})
        keys = cite.find_all('div', attrs={'class':'gsc_vcd_field'})
        values = cite.find_all('div', attrs={'class':'gsc_vcd_value'})
        for k,v in zip(keys, values):
            if k.text == 'Scholar articles':
                continue
            if k.text == 'Total citations':
                _json[k.text] = int(v.find('a').text[9:])
            else:
                _json[k.text] = v.get_text('\n')
        driver.find_element_by_xpath('//a[@id="gs_md_cita-d-x"]').click()
    except:
        continue
    if _json:
        json_str = json.dumps(_json)
        if json_str not in publications:
            publications.append(json_str)
            
publications = [json.loads(c) for c in publications]

In [21]:
with open('liling-tan.gscholar.json', 'w') as fout:
    json.dump(publications, fout)